# Symbology mapping using dedicated service - F#

### Overview
Reference data encompasses a wide range of specification data about:
* financial instrument such as asset class, symbol, maturity, etc.
* counterparties such as issuer information.
* pricing such as Open, high low and close prices.

The challenge with reference data is that it tends to be sourced from multiple sources: internal, counterparties and providers.

Reference data dedicated service is a result of various data sources and fields sourcing, validation, cross-checking and normalization.<br>

This sample requests **on-demand** instrument's identifiers and enables fields mapping by calling a dedicated static data service, making available clean data.

### Inputs/outputs
Symbology mapping sample is designed to help you searching and requesting instrument's specification data by using a wide range of commonly used fields.
* It takes instrument specification parameter(s) as per input filter and returns the *entries* matching the request.
* The returned *entries* contain `instrument identifier` data

The samples below expose several ways to query the reference data service in order to retrieve instrument identifier and fields. 
Please refer to any other scenario in order to discover how instrument identifier is used.

### Services used
This sample uses *gRPC requests* in order to retrieve reference data set of fields from the hosted service. The queried endpoint in this script are:
* *StaticDataService*: to directly retrieve reference data objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.StaticData.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*

***

# Run reference data queries

### Step 1: Install packages

In [ ]:
//#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"

In [ ]:
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.StaticData.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
let token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN")
let api = System.Environment.GetEnvironmentVariable("GRPC_APIS")

// Add token to the request header
let headers = new Metadata()
headers.Add("Authorization", $"Bearer {token}")

// Define API entry
let channel = GrpcChannel.ForAddress($"https://{api}")

In [ ]:
// Display authentication token 
token
|> display

### Step 3: Call reference data service

In [ ]:
// instantiate the service
let service = new StaticDataService.StaticDataServiceClient(channel)

### Step 4: Create and process request

#### 4.1 Introduction
The properties one can use as per filter field(s) in order to query the API for equities or futures are the following:
* *Name*, *MIC*, *Ticker*, *Contract code*, *ISIN*, *Cusip*, *market data provider IDs*, etc.

You can combine multiple filters in order to target more accurately your request. Combining filters act as a *AND* operator.

The service returns *entries* matching the request filters.
A reply contains two *arrays* dedicated for *futures* and *equities*. Keep in mind that these arrays might be empty depending on the query filters and the results.
Other shared fields such as <i>`Name`</i> or <i>`Identifier`</i> are there to make it easier to identify the instrument needed (among possible other results).  
Depending on the asset type, there might be additional fields for the aforementioned purpose.

The following code snippets will expose the different ways to query for reference data, with different asset types and various parameters.

#### 4.2 Retrieve reference data - Equities

To request for equities, the user must specify the AssetType as <i>`AssetType.Equity`</i> in the static data request.

##### 4.2.1 Equity - by Name

In [ ]:
// Generate request
let request = new StaticDataRequest(AssetType = AssetType.Equity, Name = "Apple")

let reply = service.StaticData(request, headers)
display(reply.Equities)

##### 4.2.2 Equity - by MIC code

In [ ]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    Exchange = "XNGS", 
    Count = 3, // Select the first elements
    Start = 2 // Starting from element index
  )

let reply = service.StaticData(request, headers)
display(reply.Equities)

##### 4.2.3 Equity - by Ticker

In [ ]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    Ticker = "AAPL"
  )

let reply = service.StaticData(request, headers)
display(reply.Equities)

##### 4.2.4 Equity - by Codes

In [ ]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    Code = "US0378331005" // ISIN code
  )
let reply = service.StaticData(request, headers)
display(reply.Equities)

In [ ]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    Code = "BBG000B9Y6P9" // Bloomberg Figi code
  )

let reply = service.StaticData(request, headers)
display(reply.Equities)

##### 4.2.5 Equity - by Ticker + MIC code

In [ ]:
// The service also enables to query with multiple filters as follows
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    Ticker = "AAPL",
    Exchange = "BATS"
  )

let reply = service.StaticData(request, headers)
display(reply.Equities)

##### 4.2.6 Equity - index components

In [ ]:
// Get an  index components
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    Index = "NASDAQ 100",
    Count = 10
  )

let reply = service.StaticData(request, headers)
display(reply.Equities)

##### 4.2.7 Equity - sectors

In [ ]:
// Get equities by sectors
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Equity,
    EquitySector = "Electronic Computers",
    Count = 10
  )

let reply = service.StaticData(request, headers)
display(reply.Equities)

#### 4.3 Retrieve reference data - Futures

To request for futures, the user must specify the AssetType as <i>`AssetType.Future`</i> in the static data request.

##### 4.3.1 Future - by Name

In [ ]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Future,
    Name = "WTI Crude Futures"
  )

let reply = service.StaticData(request, headers)
reply.Futures
|> Seq.sortBy (fun f -> f.Maturity.Year * 16 + f.Maturity.Month )
|> display

##### 4.3.2 Future - by Future contract

In [ ]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Future,
    FutureContract = "WBS"
  )

let reply = service.StaticData(request, headers)
reply.Futures
|> Seq.sortBy (fun f -> f.Maturity.Year * 16 + f.Maturity.Month)
|> display

##### 4.3.3 Future - by Ticker

In [ ]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Future,
    Ticker = "WBSH20"
  )

let reply = service.StaticData(request, headers)
reply.Futures
|> display

##### 4.3.4 Future - by Category

In [ ]:
let request = 
  new StaticDataRequest(
    AssetType = AssetType.Future,
    FutureCategory = "Crude Oil",
    Count = 3
  )

let reply = service.StaticData(request, headers)
reply.Futures
|> display